# Interactive Session (4)
This Jupyter notebook is provided as a companion to help you practice your learnings in the class, and answer to the questions in the Interactive Session (4). You do not need this notebook in order to finish Interactive Session (4), but this is provided as a help for you to experiment before answering the questions.

Please note that not all queries in the cells of this notebook are supposed to run properly. Some of them will fail and you are expected to find the reason for that.


We are going to use the bank.db database again. The database has six tables. The following shows their schemas. Primary key attributes are underlined and foreign keys are noted in superscript.
 - Customer = {<span style="text-decoration:underline">customerID</span>, firstName, lastName, income, birthDate}
 - Account = {<span style="text-decoration:underline">accNumber</span>, type, balance, branchNumber<sup>FK-Branch</sup>}
 - Owns = {<span style="text-decoration:underline">customerID</span><sup>FK-Customer</sup>, <span style="text-decoration:underline">accNumber</span><sup>FK-Account</sup>}
 - Transactions = {<span style="text-decoration:underline">transNumber</span>, <span style="text-decoration:underline">accNumber</span><sup>FK-Account</sup>, amount}
 - Employee = {<span style="text-decoration:underline">sin</span>, firstName, lastName, salary, branchNumber<sup>FK-Branch</sup>}
 - Branch = {<span style="text-decoration:underline">branchNumber</span>, branchName, managerSIN<sup>FK-Employee</sup>, budget}

## Initial Steps

Please run the following few cells before we start. They create the required tables and insert some tuples so that we can start experimenting with them.

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///bank.db

### Views

Let's see how many transactions exist in the Transactions table:

In [ ]:
%%sql
SELECT count(*) FROM Transactions

Now, assume we want to work frequently with transactions on the banking accounts of "Amy Hayes":

In [ ]:
%%sql
SELECT customerID, firstName, lastName, accNumber, type, balance, branchNumber, transNumber, amount FROM Customer NATURAL JOIN Account NATURAL JOIN Owns NATURAL JOIN Transactions WHERE firstName="Amy" AND lastName="Hayes"

We can define the `VIEW` below instead:

In [ ]:
%%sql
CREATE VIEW AHTransactions(customerID, firstName, lastName, accNumber, type, balance, branchNumber, transNumber, amount) AS
SELECT 
    customerID, firstName, lastName, accNumber, type, balance, branchNumber, transNumber, amount 
FROM 
    Customer NATURAL JOIN Account NATURAL JOIN Owns NATURAL JOIN Transactions 
WHERE 
    firstName="Amy" AND lastName="Hayes";

Now, instead of working with the all of the tables Customers, Accounts, Owns, and Transactions every time we need these data, we can only work with this view. 

In [ ]:
%%sql
SELECT * FROM AHTransactions;

For example, instead of:

In [ ]:
%%sql
SELECT 
    accNumber, transNumber, amount 
FROM 
    Customer NATURAL JOIN Account NATURAL JOIN Owns NATURAL JOIN Transactions 
WHERE 
    firstName="Amy" AND lastName="Hayes" AND amount>1200;

We can write:

In [ ]:
%%sql
SELECT accNumber, transNumber, amount FROM  AHTransactions WHERE amount>1200;

Now let's track changes in our relations and views.

In [ ]:
%%sql
INSERT INTO 
    Transactions (accNumber, transNumber, amount)
VALUES
    (296, 6, 1234);

In [ ]:
%%sql
SELECT * FROM AHTransactions;

In [ ]:
%%sql
DELETE FROM Transactions WHERE accNumber=296 AND transNumber=6;

In [ ]:
%%sql
SELECT * FROM AHTransactions;

Run the queries above again using `EXPLAIN QUERY PLAN`, and try to understand what has happened.

In [ ]:
%%sql
EXPLAIN QUERY PLAN
INSERT INTO 
    Transactions (accNumber, transNumber, amount)
VALUES
    (296, 6, 1234);

In [ ]:
%%sql
EXPLAIN QUERY PLAN
DELETE FROM Transactions WHERE accNumber=296 AND transNumber=6;

Now let's check changes made on `VIEWS`. SQLite `VIEWS` are read-only.

Updatable View Limitation on MySQL: 
https://www.mysqltutorial.org/create-sql-updatable-views.aspx
and https://www.sqlite.org/omitted.html 

Updatable View Limitations on PostGreSQL:
https://www.postgresqltutorial.com/postgresql-updatable-views/

Views in SQLite:
https://www.sqlitetutorial.net/sqlite-create-view/

Therefore, no changes made with the following:

In [ ]:
%%sql
INSERT INTO 
    AHTransactions (accNumber, transNumber, amount)
VALUES
    (296, 6, 1234);

In [ ]:
%%sql
SELECT * FROM AHTransactions;

and the following:

In [ ]:
%%sql
DELETE FROM AHTransactions WHERE accNumber=296 AND transNumber=6;

However, we can reflect changes on the view as changes on the initial tables using `INSTEAD OF` triggers.

In [ ]:
%%sql
CREATE TRIGGER AHTransactionsT 
    INSTEAD OF INSERT ON AHTransactions
BEGIN
    INSERT INTO Transactions(accNumber, transNumber, amount)
    VALUES(NEW.accNumber, NEW.transNumber, NEW.amount);
END;

In [ ]:
%%sql
INSERT INTO 
    AHTransactions (accNumber, transNumber, amount)
VALUES
    (296, 6, 1234);

In [ ]:
%%sql
SELECT accNumber, transNumber, amount FROM  AHTransactions WHERE amount>1200;

In [ ]:
%%sql
CREATE TRIGGER AHTransactionsDelT 
    INSTEAD OF DELETE ON AHTransactions
BEGIN
    DELETE FROM Transactions
    WHERE Transactions.accNumber==old.accNumber AND Transactions.transNumber==old.transNumber;
END;

In [ ]:
%%sql
DELETE FROM AHTransactions WHERE accNumber=296 AND transNumber=6;

Can you wrtie a trigger that on insert of new values on the AHTransactions view defined above will insert those values in to the initial transactions relation, and update the balance attribute on the accounts table?

### Clean up Steps

In [ ]:
%%sql
DROP TRIGGER AHTransactionsDelT;

In [ ]:
%%sql
DROP TRIGGER AHTransactionsT;

In [ ]:
%%sql
DROP VIEW AHTransactions;

### Indexes 

Please try the following cells, then answer corresponding questions in the IS(4) Canvas Quiz.

In [ ]:
%%sql
SELECT * FROM sqlite_master WHERE type = 'index';

In [ ]:
%%sql
EXPLAIN QUERY PLAN
SELECT * FROM Customer WHERE firstName="Amy";

In [ ]:
%%sql
CREATE INDEX MyCustomerIndex ON Customer(firstName);

In [ ]:
%%sql
CREATE INDEX MyCustomerLNIndex ON Customer(lastName);

In [ ]:
%%sql
CREATE INDEX MyCustomerFLNIndex ON Customer(firstName,lastName);

In [ ]:
%%sql
EXPLAIN QUERY PLAN
SELECT * FROM Customer WHERE firstName="Amy";

Sometimes we can delete and recreated our indexes using REINDEX:

In [ ]:
%%sql
REINDEX

Please find out more about REINDEX [here](https://www.sqlite.org/lang_reindex.html).

### Clean up Steps

In [ ]:
%%sql
DROP INDEX MyCustomerIndex;

In [ ]:
%%sql
DROP INDEX MyCustomerLNIndex;

In [ ]:
%%sql
DROP INDEX MyCustomerFLNIndex;